In [2]:
import cPickle
import numpy as np
from collections import defaultdict, OrderedDict
import theano
import theano.tensor as T
import re
import warnings
import pandas as pd
import sys
import time
warnings.filterwarnings("ignore")   


def ReLU(x):
    y = T.maximum(0.0, x)
    return(y)
def Sigmoid(x):
    y = T.nnet.sigmoid(x)
    return(y)
def Tanh(x):
    y = T.tanh(x)
    return(y)
def Iden(x):
    y = x
    return(y)
       
def train_conv_net(datasets, U, img_w=300, filter_hs=[3,4,5], hidden_units=[100,2],  dropout_rate=[0.5], shuffle_batch=True,
                   n_epochs=25, batch_size=50, lr_decay = 0.95, conv_non_linear="relu", activations=[Iden], sqr_norm_lim=9,
                   non_static=True):
    """
    Train a simple conv net
    img_h = sentence length (padded where necessary)
    img_w = word vector length (300 for word2vec)
    filter_hs = filter window sizes    
    hidden_units = [x,y] x is the number of feature maps (per filter window), and y is the penultimate layer
    sqr_norm_lim = s^2 in the paper
    lr_decay = adadelta decay parameter
    """    
    rng = np.random.RandomState(3435)
    img_h = len(datasets[0][0])-1  
    filter_w = img_w    
    feature_maps = hidden_units[0]
    filter_shapes = []
    pool_sizes = []
    for filter_h in filter_hs:
        filter_shapes.append((feature_maps, 1, filter_h, filter_w))
        pool_sizes.append((img_h-filter_h+1, img_w-filter_w+1))
    parameters = [("image shape",img_h,img_w),("filter shape",filter_shapes), ("hidden_units",hidden_units),
                  ("dropout", dropout_rate), ("batch_size",batch_size),("non_static", non_static),
                    ("learn_decay",lr_decay), ("conv_non_linear", conv_non_linear), ("non_static", non_static)
                    ,("sqr_norm_lim",sqr_norm_lim),("shuffle_batch",shuffle_batch)]
    print parameters    
    
    #define model architecture
    index = T.lscalar()
    x = T.matrix('x')   
    y = T.ivector('y')
    Words = theano.shared(value = U, name = "Words")
    zero_vec_tensor = T.vector()
    zero_vec = np.zeros(img_w)
    set_zero = theano.function([zero_vec_tensor], updates=[(Words, T.set_subtensor(Words[0,:], 
                                zero_vec_tensor))], allow_input_downcast=True)
    layer0_input = Words[T.cast(x.flatten(),dtype="int32")].reshape((x.shape[0],1,x.shape[1],
                                                                     Words.shape[1]))                                  
    conv_layers = []
    layer1_inputs = []
    for i in xrange(len(filter_hs)):
        filter_shape = filter_shapes[i]
        pool_size = pool_sizes[i]
        conv_layer = LeNetConvPoolLayer(rng, input=layer0_input,image_shape=(batch_size, 1, img_h, img_w),
                                filter_shape=filter_shape, poolsize=pool_size, non_linear=conv_non_linear)
        layer1_input = conv_layer.output.flatten(2)
        conv_layers.append(conv_layer)
        layer1_inputs.append(layer1_input)
    layer1_input = T.concatenate(layer1_inputs,1)
    hidden_units[0] = feature_maps*len(filter_hs)    
    classifier = MLPDropout(rng, input=layer1_input, layer_sizes=hidden_units, activations=activations, 
                            dropout_rates=dropout_rate)
    
    #define parameters of the model and update functions using adadelta
    params = classifier.params     
    for conv_layer in conv_layers:
        params += conv_layer.params
    if non_static:
        #if word vectors are allowed to change, add them as model parameters
        params += [Words]
    cost = classifier.negative_log_likelihood(y) 
    dropout_cost = classifier.dropout_negative_log_likelihood(y)           
    grad_updates = sgd_updates_adadelta(params, dropout_cost, lr_decay, 1e-6, sqr_norm_lim)
    
    #shuffle dataset and assign to mini batches. if dataset size is not a multiple of mini batches, replicate 
    #extra data (at random)
    np.random.seed(3435)
    if datasets[0].shape[0] % batch_size > 0:
        extra_data_num = batch_size - datasets[0].shape[0] % batch_size
        train_set = np.random.permutation(datasets[0])   
        extra_data = train_set[:extra_data_num]
        new_data=np.append(datasets[0],extra_data,axis=0)
    else:
        new_data = datasets[0]
    new_data = np.random.permutation(new_data)
    
    #n_batches = new_data.shape[0]/batch_size
    #n_train_batches = int(np.round(n_batches*0.9))
    
    n_batches = new_data.shape[0]/batch_size
    n_train_batches = n_batches
   
    train_set = new_data[:,:]
    val_set = datasets[1]
    test_set_x = datasets[2]

    
    train_set_x, train_set_y = shared_dataset((train_set[:,:img_h],train_set[:,-1]))
    val_set_x, val_set_y = shared_dataset((val_set[:,:img_h],val_set[:,-1]))
    #n_val_batches = n_batches - n_train_batches
    n_val_batches = int(np.round(val_set.shape[0]/batch_size))
    
    get_acc_val_model = theano.function([index], classifier.errors(y),
        givens={
            x: val_set_x[index * batch_size: (index + 1) * batch_size],
                y: val_set_y[index * batch_size: (index + 1) * batch_size]},
                                allow_input_downcast=True)
            
    #compile theano functions to get train/val/test errors
    get_acc_train_model = theano.function([index], classifier.errors(y),
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
                y: train_set_y[index * batch_size: (index + 1) * batch_size]},
                                allow_input_downcast=True)               
    train_model = theano.function([index], cost, updates=grad_updates,
        givens={
            x: train_set_x[index*batch_size:(index+1)*batch_size],
                y: train_set_y[index*batch_size:(index+1)*batch_size]},
                                allow_input_downcast = True)     
    test_pred_layers_a = []
    test_pred_layers_b = []
    
    
    # If test set is large loading the whole test set can give GPU memory allocation error
    # So we make prediction only by taking a maximum of 2000 test examples at a time
    test_size = test_set_x.shape[0]
    test_batch_size = 2000
    test_iter = int(test_size/test_batch_size)
    extra_test_size = test_size - test_iter * test_batch_size
    
    test_layer0_input_a = Words[T.cast(x.flatten(),dtype="int32")].reshape((test_batch_size,1,img_h,Words.shape[1]))
    for conv_layer in conv_layers:
        test_layer0_output_a = conv_layer.predict(test_layer0_input_a, test_batch_size)
        test_pred_layers_a.append(test_layer0_output_a.flatten(2))
    test_layer1_input_a = T.concatenate(test_pred_layers_a, 1)
    test_y_pred_a = classifier.predict(test_layer1_input_a)
    
    
    test_layer0_input_b = Words[T.cast(x.flatten(),dtype="int32")].reshape((extra_test_size,1,img_h,Words.shape[1]))
    for conv_layer in conv_layers:
        test_layer0_output_b = conv_layer.predict(test_layer0_input_b, extra_test_size)
        test_pred_layers_b.append(test_layer0_output_b.flatten(2))
    test_layer1_input_b = T.concatenate(test_pred_layers_b, 1)
    test_y_pred_b = classifier.predict(test_layer1_input_b)
    
    
    test_model_all_a = theano.function([x], test_y_pred_a, allow_input_downcast = True)   
    test_model_all_b = theano.function([x], test_y_pred_b, allow_input_downcast = True) 
    
    #start training over mini-batches
    print '... training'
    epoch = 0
    best_val_perf = 0
    val_perf = 0
    test_perf = 0       
    cost_epoch = 0  
    while (epoch < n_epochs):
        start_time = time.time()
        epoch = epoch + 1
        if shuffle_batch:
            for minibatch_index in np.random.permutation(range(n_train_batches)):
                cost_epoch = train_model(minibatch_index)
                set_zero(zero_vec)
        else:
            for minibatch_index in xrange(n_train_batches):
                cost_epoch = train_model(minibatch_index)  
                set_zero(zero_vec)
        train_losses = [get_acc_train_model(i) for i in xrange(n_train_batches)]
        train_perf = 1 - np.mean(train_losses)
        val_losses = [get_acc_val_model(i) for i in xrange(n_val_batches)]
        val_perf = 1- np.mean(val_losses)                        
        print('epoch: %i, training time: %.2f secs, train perf: %.2f %%, val perf: %.2f %%' % (epoch, 
                                         time.time()-start_time, train_perf * 100., val_perf*100.))

    prediction1 = np.zeros((test_iter, test_batch_size))
    for j in xrange(test_iter):
        prediction1[j] = test_model_all_a(test_set_x[test_batch_size*j:test_batch_size*(j+1) , :])
        
    prediction2 = test_model_all_b(test_set_x[-extra_test_size: , :])    
    prediction = list(prediction1.reshape(test_batch_size * test_iter)) + list(prediction2)
    return prediction
        
def shared_dataset(data_xy, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        data_x, data_y = data_xy
        shared_x = theano.shared(np.asarray(data_x,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        return shared_x, T.cast(shared_y, 'int32')
        
def sgd_updates_adadelta(params,cost,rho=0.95,epsilon=1e-6,norm_lim=9,word_vec_name='Words'):
    """
    adadelta update rule, mostly from
    https://groups.google.com/forum/#!topic/pylearn-dev/3QbKtCumAW4 (for Adadelta)
    """
    updates = OrderedDict({})
    exp_sqr_grads = OrderedDict({})
    exp_sqr_ups = OrderedDict({})
    gparams = []
    for param in params:
        empty = np.zeros_like(param.get_value())
        exp_sqr_grads[param] = theano.shared(value=as_floatX(empty),name="exp_grad_%s" % param.name)
        gp = T.grad(cost, param)
        exp_sqr_ups[param] = theano.shared(value=as_floatX(empty), name="exp_grad_%s" % param.name)
        gparams.append(gp)
    for param, gp in zip(params, gparams):
        exp_sg = exp_sqr_grads[param]
        exp_su = exp_sqr_ups[param]
        up_exp_sg = rho * exp_sg + (1 - rho) * T.sqr(gp)
        updates[exp_sg] = up_exp_sg
        step =  -(T.sqrt(exp_su + epsilon) / T.sqrt(up_exp_sg + epsilon)) * gp
        updates[exp_su] = rho * exp_su + (1 - rho) * T.sqr(step)
        stepped_param = param + step
        if (param.get_value(borrow=True).ndim == 2) and (param.name!='Words'):
            col_norms = T.sqrt(T.sum(T.sqr(stepped_param), axis=0))
            desired_norms = T.clip(col_norms, 0, T.sqrt(norm_lim))
            scale = desired_norms / (1e-7 + col_norms)
            updates[param] = stepped_param * scale
        else:
            updates[param] = stepped_param      
    return updates 

def as_floatX(variable):
    if isinstance(variable, float):
        return np.cast[theano.config.floatX](variable)

    if isinstance(variable, np.ndarray):
        return np.cast[theano.config.floatX](variable)
    return theano.tensor.cast(variable, theano.config.floatX)
    
def safe_update(dict_to, dict_from):
    """
    re-make update dictionary for safe updating
    """
    for key, val in dict(dict_from).iteritems():
        if key in dict_to:
            raise KeyError(key)
        dict_to[key] = val
    return dict_to
    
def get_idx_from_sent(sent, word_idx_map, max_l=51, k=300, filter_h=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    pad = filter_h - 1
    for i in xrange(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l+2*pad:
        x.append(0)
    return x

def make_idx_train_valid_data(train_revs, word_idx_map, cv, max_l = 51, k = 300, filter_h = 5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, valid = [], []
    for rev in train_revs:
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h)   
        sent.append(rev["y"])
        if rev["split"]==cv:            
            valid.append(sent)        
        else:  
            train.append(sent)
 
    train = np.array(train,dtype="int")
    valid = np.array(valid,dtype='int' )
    return [train, valid]    

def make_idx_data1(revs, word_idx_map, max_l = 51, k = 300, filter_h = 5):
    """
    Transforms sentences into a 2-d matrix.
    """
    test = []
    for rev in revs:
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h) 
        sent.append(rev["y"])
        test.append(sent)  
        
    test = np.array(test,dtype="int")
    return [test]  

def make_idx_data2(revs, word_idx_map, max_l = 51, k = 300, filter_h = 5):
    """
    Transforms sentences into a 2-d matrix.
    """
    test = []
    for rev in revs:
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h) 
        sent.append(rev["y"])
        test.append(sent)  
        
    test = np.array(test,dtype="int")
    return test

def make_idx_data3(revs, word_idx_map, max_l = 51, k = 300, filter_h = 5):
    """
    Transforms sentences into a 2-d matrix.
    """
    test = []
    for rev in revs:
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h)   
        test.append(sent)  
        
    test = np.array(test,dtype="int")
    return test

Using gpu device 0: GeForce GT 755M (CNMeM is enabled with initial size: 80.0% of memory, CuDNN not available)


In [3]:
print 'Loading data...',
x = cPickle.load(open("mr.p35","rb"))
train_revs,valid_revs,test_revs,W1,W2,W3,W4,W5,W6,word_idx_map,vocab = x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10]
print "Data loaded!"

Loading data... Data loaded!


In [4]:
train_csv = pd.DataFrame.from_csv('all_train1.csv', index_col=False)
valid_csv = pd.DataFrame.from_csv('valid.csv'  , index_col=False)
test_csv  = pd.DataFrame.from_csv('test.csv'   , index_col=False)
datasets  = make_idx_data1(train_revs, word_idx_map, max_l = 53, k = 300, filter_h = 5)
valid     = make_idx_data2(valid_revs, word_idx_map, max_l = 53, k = 300, filter_h = 5)
test      = make_idx_data3(test_revs , word_idx_map, max_l = 53, k = 300, filter_h = 5)

In [5]:
datasets.append(valid)
datasets.append(test)
y_test = np.array(test_csv['label'])

In [6]:
execfile("conv_net_classes.py")

In [8]:
score1 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W1, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score1[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score1[i]
print 'Test Accuracy ' + ' : ' + str(score1.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', True), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', True), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 164.38 secs, train perf: 90.31 %, val perf: 83.88 %
epoch: 2, training time: 164.35 secs, train perf: 88.80 %, val perf: 81.53 %
epoch: 3, training time: 164.35 secs, train perf: 94.92 %, val perf: 84.00 %
epoch: 4, training time: 164.89 secs, train perf: 95.82 %, val perf: 85.65 %
epoch: 5, training time: 164.64 secs, train perf: 96.38 %, val perf: 85.53 %
epoch: 6, training time: 164.65 secs, train perf: 96.84 %, val perf: 85.65 %
epoch: 7, training time: 164.48 secs, train perf: 97.22 %, val perf: 84.71 %
epoch: 8, training time: 164.47 secs, train perf: 97.44 %, val perf: 84.59 %
epoch: 9, training time: 165.67 secs, train perf: 97.64 %, v

In [9]:
score2 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W2, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score2[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score2[i]
print 'Test Accuracy ' + ' : ' + str(score2.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', True), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', True), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 172.90 secs, train perf: 90.54 %, val perf: 84.82 %
epoch: 2, training time: 173.09 secs, train perf: 90.76 %, val perf: 82.94 %
epoch: 3, training time: 172.91 secs, train perf: 95.32 %, val perf: 85.18 %
epoch: 4, training time: 172.85 secs, train perf: 96.06 %, val perf: 85.41 %
epoch: 5, training time: 172.96 secs, train perf: 96.54 %, val perf: 84.94 %
epoch: 6, training time: 172.87 secs, train perf: 97.05 %, val perf: 86.12 %
epoch: 7, training time: 172.86 secs, train perf: 97.43 %, val perf: 84.82 %
epoch: 8, training time: 172.73 secs, train perf: 97.63 %, val perf: 85.18 %
epoch: 9, training time: 172.86 secs, train perf: 97.84 %, v

In [10]:
score3 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W3, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score3[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score3[i]
print 'Test Accuracy ' + ' : ' + str(score3.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', True), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', True), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 164.83 secs, train perf: 91.07 %, val perf: 84.59 %
epoch: 2, training time: 172.54 secs, train perf: 91.38 %, val perf: 83.18 %
epoch: 3, training time: 174.73 secs, train perf: 95.43 %, val perf: 85.88 %
epoch: 4, training time: 173.95 secs, train perf: 96.11 %, val perf: 85.29 %
epoch: 5, training time: 170.04 secs, train perf: 96.60 %, val perf: 85.29 %
epoch: 6, training time: 164.21 secs, train perf: 96.96 %, val perf: 85.65 %
epoch: 7, training time: 164.24 secs, train perf: 97.49 %, val perf: 83.76 %
epoch: 8, training time: 164.18 secs, train perf: 97.66 %, val perf: 85.18 %
epoch: 9, training time: 164.13 secs, train perf: 97.77 %, v

In [11]:
score4 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W4, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score4[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score4[i]
print 'Test Accuracy ' + ' : ' + str(score4.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', True), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', True), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 166.12 secs, train perf: 91.35 %, val perf: 84.00 %
epoch: 2, training time: 168.17 secs, train perf: 93.01 %, val perf: 84.47 %
epoch: 3, training time: 168.22 secs, train perf: 94.74 %, val perf: 84.35 %
epoch: 4, training time: 168.14 secs, train perf: 96.18 %, val perf: 85.41 %
epoch: 5, training time: 168.13 secs, train perf: 96.71 %, val perf: 85.76 %
epoch: 6, training time: 168.80 secs, train perf: 97.08 %, val perf: 85.65 %
epoch: 7, training time: 168.89 secs, train perf: 97.52 %, val perf: 84.71 %
epoch: 8, training time: 168.49 secs, train perf: 97.78 %, val perf: 85.65 %
epoch: 9, training time: 167.72 secs, train perf: 97.88 %, v

In [10]:
score5 = [0,86.71,86.82]
for i in xrange(1):
    y_prediction = train_conv_net(datasets, W5, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score5[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score5[i]
print 'Test Accuracy ' + ' : ' + str(score5.mean()) + ' %'
print score5
print np.array(score5).mean()

[86.326194398682048, 86.71, 86.82]
86.6187314662


In [11]:
score6 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W6, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=True, batch_size=50,
                                  dropout_rate=[0.5])
    
    score6[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score6[i]
print 'Test Accuracy ' + ' : ' + str(score6.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', True), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', True), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 164.95 secs, train perf: 91.34 %, val perf: 84.82 %
epoch: 2, training time: 164.85 secs, train perf: 92.63 %, val perf: 84.12 %
epoch: 3, training time: 164.80 secs, train perf: 95.02 %, val perf: 84.82 %
epoch: 4, training time: 164.87 secs, train perf: 96.10 %, val perf: 85.88 %
epoch: 5, training time: 173.25 secs, train perf: 96.82 %, val perf: 85.06 %
epoch: 6, training time: 178.13 secs, train perf: 97.05 %, val perf: 86.59 %
epoch: 7, training time: 177.55 secs, train perf: 97.54 %, val perf: 84.94 %
epoch: 8, training time: 177.89 secs, train perf: 97.79 %, val perf: 85.53 %
epoch: 9, training time: 178.09 secs, train perf: 97.83 %, v

In [6]:
score1 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W1, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score1[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score1[i]
print 'Test Accuracy ' + ' : ' + str(score1.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 90.29 secs, train perf: 87.23 %, val perf: 83.41 %
epoch: 2, training time: 90.53 secs, train perf: 85.51 %, val perf: 82.24 %
epoch: 3, training time: 90.49 secs, train perf: 91.09 %, val perf: 84.59 %
epoch: 4, training time: 90.36 secs, train perf: 92.31 %, val perf: 84.59 %
epoch: 5, training time: 90.65 secs, train perf: 93.69 %, val perf: 84.71 %
epoch: 6, training time: 90.48 secs, train perf: 94.43 %, val perf: 84.35 %
epoch: 7, training time: 90.21 secs, train perf: 94.22 %, val perf: 82.94 %
epoch: 8, training time: 90.19 secs, train perf: 95.21 %, val perf: 84.00 %
epoch: 9, training time: 90.18 secs, train perf: 95.57 %, val perf

In [7]:
score2 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W2, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score2[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score2[i]
print 'Test Accuracy ' + ' : ' + str(score2.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 105.94 secs, train perf: 87.82 %, val perf: 84.00 %
epoch: 2, training time: 90.83 secs, train perf: 86.03 %, val perf: 81.06 %
epoch: 3, training time: 91.22 secs, train perf: 92.32 %, val perf: 84.35 %
epoch: 4, training time: 90.74 secs, train perf: 93.39 %, val perf: 83.41 %
epoch: 5, training time: 90.73 secs, train perf: 94.21 %, val perf: 84.35 %
epoch: 6, training time: 90.42 secs, train perf: 94.89 %, val perf: 84.47 %
epoch: 7, training time: 90.42 secs, train perf: 95.37 %, val perf: 83.88 %
epoch: 8, training time: 90.91 secs, train perf: 95.77 %, val perf: 84.00 %
epoch: 9, training time: 91.22 secs, train perf: 95.77 %, val per

In [8]:
score3 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W3, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score3[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score3[i]
print 'Test Accuracy ' + ' : ' + str(score3.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 91.17 secs, train perf: 88.41 %, val perf: 84.24 %
epoch: 2, training time: 91.15 secs, train perf: 86.87 %, val perf: 81.06 %
epoch: 3, training time: 91.13 secs, train perf: 92.46 %, val perf: 83.65 %
epoch: 4, training time: 91.16 secs, train perf: 93.81 %, val perf: 85.41 %
epoch: 5, training time: 91.21 secs, train perf: 94.79 %, val perf: 85.18 %
epoch: 6, training time: 91.24 secs, train perf: 95.22 %, val perf: 83.88 %
epoch: 7, training time: 91.25 secs, train perf: 95.33 %, val perf: 83.65 %
epoch: 8, training time: 91.71 secs, train perf: 95.90 %, val perf: 84.35 %
epoch: 9, training time: 99.86 secs, train perf: 96.20 %, val perf

In [ ]:
score4 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W4, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score4[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score4[i]
print 'Test Accuracy ' + ' : ' + str(score4.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 91.21 secs, train perf: 88.42 %, val perf: 83.53 %
epoch: 2, training time: 91.09 secs, train perf: 89.98 %, val perf: 83.53 %
epoch: 3, training time: 91.14 secs, train perf: 92.61 %, val perf: 84.47 %
epoch: 4, training time: 93.12 secs, train perf: 94.04 %, val perf: 84.94 %
epoch: 5, training time: 98.87 secs, train perf: 94.69 %, val perf: 84.24 %
epoch: 6, training time: 98.49 secs, train perf: 95.33 %, val perf: 84.24 %
epoch: 7, training time: 92.76 secs, train perf: 95.75 %, val perf: 83.65 %
epoch: 8, training time: 90.50 secs, train perf: 96.33 %, val perf: 83.76 %
epoch: 9, training time: 90.50 secs, train perf: 96.45 %, val perf

In [7]:
score5 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W5, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score5[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score5[i]
print 'Test Accuracy ' + ' : ' + str(score5.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 90.34 secs, train perf: 88.58 %, val perf: 83.76 %
epoch: 2, training time: 90.51 secs, train perf: 89.16 %, val perf: 83.18 %
epoch: 3, training time: 90.50 secs, train perf: 90.60 %, val perf: 82.82 %
epoch: 4, training time: 90.48 secs, train perf: 93.71 %, val perf: 84.00 %
epoch: 5, training time: 90.47 secs, train perf: 94.76 %, val perf: 84.12 %
epoch: 6, training time: 90.50 secs, train perf: 95.48 %, val perf: 82.94 %
epoch: 7, training time: 90.88 secs, train perf: 95.34 %, val perf: 83.65 %
epoch: 8, training time: 90.98 secs, train perf: 96.29 %, val perf: 84.12 %
epoch: 9, training time: 90.88 secs, train perf: 96.52 %, val perf

In [8]:
score6 = np.zeros(3)
for i in xrange(3):
    y_prediction = train_conv_net(datasets, W6, lr_decay=0.95, filter_hs=[3,4,5], conv_non_linear="relu", hidden_units=[100,2], 
                                  shuffle_batch=True, n_epochs=10, sqr_norm_lim=3, non_static=False, batch_size=50,
                                  dropout_rate=[0.5])
    
    score6[i] = np.sum(y_test == y_prediction, axis = 0) * 100/float(len(y_test))
    print score6[i]
print 'Test Accuracy ' + ' : ' + str(score6.mean()) + ' %'

[('image shape', 61, 300), ('filter shape', [(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]), ('hidden_units', [100, 2]), ('dropout', [0.5]), ('batch_size', 50), ('non_static', False), ('learn_decay', 0.95), ('conv_non_linear', 'relu'), ('non_static', False), ('sqr_norm_lim', 3), ('shuffle_batch', True)]
... training
epoch: 1, training time: 91.01 secs, train perf: 88.81 %, val perf: 84.47 %
epoch: 2, training time: 90.92 secs, train perf: 88.11 %, val perf: 82.47 %
epoch: 3, training time: 90.93 secs, train perf: 92.00 %, val perf: 84.35 %
epoch: 4, training time: 90.88 secs, train perf: 93.95 %, val perf: 85.18 %
epoch: 5, training time: 97.28 secs, train perf: 94.81 %, val perf: 85.76 %
epoch: 6, training time: 99.61 secs, train perf: 95.42 %, val perf: 85.18 %
epoch: 7, training time: 99.51 secs, train perf: 95.43 %, val perf: 84.00 %
epoch: 8, training time: 99.14 secs, train perf: 96.32 %, val perf: 84.59 %
epoch: 9, training time: 99.11 secs, train perf: 96.65 %, val perf